In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')#change dir

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing import image #// test checl
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
from keras.applications.resnet import ResNet101
from keras.applications.resnet50 import ResNet50

Using TensorFlow backend.


In [3]:
df_train = pd.read_excel('/content/drive/My Drive/Labels_1.xlsx')
df_train.head()

,ImageName,Level
0,scen10_cam200095.jpg,1
1,scen10_cam200096.jpg,1
2,scen10_cam200097.jpg,1
3,scen10_cam200098.jpg,1
4,scen10_cam200099.jpg,1


In [0]:
targets_series = pd.Series(df_train['Level'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [0]:
one_hot_labels = np.asarray(one_hot)

In [0]:
im_size1 = 224 #deafult is 224 /given is 256 
im_size2 = 224 #deafult is 224 /given is 256 
# im_size1 = 256 #change size of 256
# im_size2 = 256 #change size of 256

In [0]:
x_train = []
y_train = []
x_test = []

In [0]:
# i = 0 
# #for folder in range(9):
# for f, breed in tqdm(df_train.values[:6786]):
#   if type(cv2.imread('Train/{}'.format(f)))==type(None):
#     continue 
#   else: 
#     img = cv2.imread('Train/{}'.format(f)) 
#     label = one_hot_labels[i] 
#     x_train.append(cv2.resize(img, (im_size1, im_size2))) 
#     y_train.append(label) 
#     i += 1 
# np.save('x_trainfinaltest',x_train) 
# np.save('y_trainfinaltest',y_train) 
# print('Done')

In [0]:
# x_train = np.load('x_trainfinaltest.npy')
# y_train = np.load('y_trainfinaltest.npy')
x_train = np.load('x_train2.npy')
y_train = np.load('y_train2.npy')

In [0]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.

In [11]:
print(x_train_raw.shape)
print(y_train_raw.shape)

(6785, 224, 224, 3)
(6785, 2)


In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.1, random_state=1)

In [0]:
num_class = y_train_raw.shape[1]

In [0]:
# base_model = ResNet50(weights = None, include_top=False, input_shape=(im_size1, im_size2, 3))

# # Add a new top layer
# x = base_model.output
# x = Flatten()(x)
# x = Dropout(0.2)(x)
# # x = Dense(32, activation='relu')(x)
# # x = Dense(16, activation='relu')(x)
# predictions = Dense(num_class, activation='softmax')(x)
# #predictions = Dense(num_class, activation='sigmoid')(x)

# # This is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)

# # First: train only the top layers (which were randomly initialized)
# #for layer in base_model.layers:
# #    layer.trainable = False
# # model.compile(loss='categorical_crossentropy', 
# #               optimizer='Adagrad', 
# #               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='sgd', 
#               metrics=['accuracy'])

# # model.compile(loss='categorical_crossentropy', 
# #               optimizer='rmsprop', 
# #               metrics=['accuracy'])

# # from tensorflow.keras.optimizers import SGD
# # model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

# callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
# model.summary()

In [0]:
# model.fit(X_train, Y_train,  epochs=10, validation_data=(X_valid, Y_valid), verbose=1)

In [16]:
base_model = ResNet101(weights = None, include_top=False, input_shape=(im_size1, im_size2, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.2)(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', 
              optimizer='sgd', 
              metrics=['accuracy'])


# model.compile(loss='categorical_crossentropy', 
#               optimizer='rmsprop', 
#               metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [17]:
model.fit(X_train, Y_train,  epochs=10, validation_data=(X_valid, Y_valid), verbose=1)

Train on 6106 samples, validate on 679 samples
Epoch 1/10
6106/6106 [==============================] - 146s 24ms/step - loss: 14.7136 - accuracy: 0.5563 - val_loss: 76.3837 - val_accuracy: 0.5758
Epoch 2/10
6106/6106 [==============================] - 107s 17ms/step - loss: 2.6427 - accuracy: 0.6271 - val_loss: 0.6238 - val_accuracy: 0.6510
Epoch 3/10
6106/6106 [==============================] - 107s 17ms/step - loss: 3.9643 - accuracy: 0.6209 - val_loss: 27756.7171 - val_accuracy: 0.5700
Epoch 4/10
6106/6106 [==============================] - 107s 17ms/step - loss: 1.8592 - accuracy: 0.6492 - val_loss: 0.6216 - val_accuracy: 0.6804
Epoch 5/10
6106/6106 [==============================] - 107s 17ms/step - loss: 2.7505 - accuracy: 0.6777 - val_loss: 2.6275 - val_accuracy: 0.5700
Epoch 6/10
6106/6106 [==============================] - 107s 18ms/step - loss: 3.4645 - accuracy: 0.6618 - val_loss: 1.7182 - val_accuracy: 0.4816
Epoch 7/10
6106/6106 [==============================] - 107s 18ms